In [17]:
from keras.layers import Input, Embedding, Convolution1D, MaxPooling1D, Flatten, Dense, Dropout, merge, concatenate
#from keras.layers import concatenate
from keras.models import Model
from keras.utils import np_utils
from keras import backend as K

import tensorflow as tf
import numpy as np
from utilities import my_callbacks
from utilities import data_helper
import optparse
import sys
#from keras.callbacks import History
#history = History()

np.set_printoptions(threshold=np.nan)

In [2]:
vocab = data_helper.load_all(filelist="final_data/wsj.all")
print(vocab)

Using features: None
{'S': 52415, 'O': 30440, 'X': 162058, '-': 2449404}
Total vocabulary size in the whole dataset: 4
['-', 'O', 'S', 'X', '0']


In [3]:
print("loading entity-gird for pos and neg documents...")

X_train_1, X_train_0, E = data_helper.load_and_numberize_Egrid_with_Feats("final_data/wsj.train", 
        perm_num = 20, maxlen=2000, window_size=6, vocab_list=vocab, emb_size=100)

X_dev_1, X_dev_0, E    = data_helper.load_and_numberize_Egrid_with_Feats("final_data/wsj.dev", 
        perm_num = 20, maxlen=2000, window_size=6, E = E, vocab_list=vocab, emb_size=100)

X_test_1, X_test_0, E    = data_helper.load_and_numberize_Egrid_with_Feats("final_data/wsj.test", 
        perm_num = 20, maxlen=2000, window_size=6, E = E, vocab_list=vocab, emb_size=100)

loading entity-gird for pos and neg documents...


In [5]:
E

array([[  7.03970985e-03,  -8.52192802e-03,   7.89863525e-03,
         -1.27012905e-03,  -7.44644532e-03,   1.51715749e-03,
          6.80941839e-03,  -1.29758898e-03,   3.91821121e-03,
          3.69276194e-03,   4.01296731e-03,   5.59388526e-03,
          2.85498737e-03,   9.22052336e-03,  -7.83070229e-03,
          5.92212678e-03,   6.65160166e-03,  -4.67983274e-03,
          6.73370778e-03,   6.42538131e-04,   3.38151282e-04,
         -8.02824575e-03,   8.37737989e-03,   3.33316984e-03,
         -6.50441043e-03,  -5.64616975e-03,  -6.42494388e-04,
         -1.28217516e-03,   7.78708962e-03,  -5.54801458e-03,
          1.78038732e-03,  -4.45596859e-03,   5.14443672e-04,
         -4.81285778e-03,   5.78972608e-04,  -3.75718510e-03,
          8.83244960e-04,  -5.15886991e-03,  -8.11523952e-03,
         -6.21067241e-03,  -6.99429344e-03,   7.88893673e-03,
         -3.98495804e-03,  -4.54271061e-03,  -9.87040510e-03,
          1.96026909e-03,   5.88701763e-03,   1.97242136e-03,
        

In [6]:
num_train = len(X_train_1)
num_dev   = len(X_dev_1)
num_test  = len(X_test_1)
#assign Y value
y_train_1 = [1] * num_train 
y_dev_1 = [1] * num_dev 
y_test_1 = [1] * num_test 

print('.....................................')
print("Num of traing pairs: " + str(num_train))
print("Num of dev pairs: " + str(num_dev))
print("Num of test pairs: " + str(num_test))
#print("Num of permutation in train: " + str(opts.p_num)) 
#print("The maximum in length for CNN: " + str(opts.maxlen))
print('.....................................')

.....................................
Num of traing pairs: 23744
Num of dev pairs: 2678
Num of test pairs: 20411
.....................................


In [7]:
# One hot encoding of the outputs
y_train_1 = np_utils.to_categorical(y_train_1, 2)
y_dev_1 = np_utils.to_categorical(y_dev_1, 2)
y_test_1 = np_utils.to_categorical(y_test_1, 2)

#randomly shuffle the training data
np.random.seed(113)
np.random.shuffle(X_train_1)
np.random.seed(113)
np.random.shuffle(X_train_0)

In [9]:
"""
np.random.seed(113)

E = 0.01 * np.random.uniform( -1.0, 1.0, (len(vocab), 100))
E[len(vocab)-1] = 0
E
"""

'\nnp.random.seed(113)\n\nE = 0.01 * np.random.uniform( -1.0, 1.0, (len(vocab), 100))\nE[len(vocab)-1] = 0\nE\n'

In [10]:
def ranking_loss(y_true, y_pred):
    pos = y_pred[:, 0]
    neg = y_pred[:, 1]
    #loss = -K.sigmoid(pos-neg) # use 
    loss = K.maximum(1.0 + neg - pos, 0.0) #if you want to use margin ranking loss
    return K.mean(loss) + 0 * y_true

In [11]:
# first, define a CNN model for sequence of entities 
sent_input = Input(shape=(2000,), dtype='int32', name='sent_input')

# embedding layer encodes the input into sequences of 300-dimenstional vectors. 
E = np.float32(E) #E was float64 which doesn't work for tensorflow conv1d function
x = Embedding(input_dim=len(vocab), output_dim=100, weights= [E], input_length=2000)(sent_input)
#print(x)

# add a convolutiaon 1D layer
#x = Dropout(dropout_ratio)(x)
x = Convolution1D(filters=150, kernel_size=6, padding='valid', activation='relu')(x)
#print(x)

# add max pooling layers
#x = AveragePooling1D(pool_length=pool_length)(x)
x = MaxPooling1D(pool_size=6)(x)
#print(x)

#x = Dropout(opts.dropout_ratio)(x)
x = Flatten()(x)
#print(x)

#x = Dense(hidden_size, activation='relu')(x)
x = Dropout(0.5, seed = 100)(x)
#print(x)

# add latent coherence score
out_x = Dense(1, activation=None)(x)
#print(out_x)

shared_cnn = Model(sent_input, out_x)
#print(shared_cnn)

In [12]:
print(shared_cnn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sent_input (InputLayer)      (None, 2000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 2000, 100)         500       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1995, 150)         90150     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 332, 150)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 49800)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 49800)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 49801     
Total para

In [13]:
# Inputs of pos and neg document
pos_input = Input(shape=(2000,), dtype='int32', name="pos_input")
neg_input = Input(shape=(2000,), dtype='int32', name="neg_input")


# these two models will share eveything from shared_cnn
pos_branch = shared_cnn(pos_input)
neg_branch = shared_cnn(neg_input)



concatenated = merge([pos_branch, neg_branch], mode='concat', name="coherence_out")
#concatenated = concatenate([pos_branch, neg_branch], name="coherence_out")
# output is two latent coherence score


final_model = Model([pos_input, neg_input], concatenated)


#final_model.compile(loss='ranking_loss', optimizer='adam')
final_model.compile(loss={'coherence_out': ranking_loss}, optimizer="rmsprop")

# setting callback
histories = my_callbacks.Histories()

#print(shared_cnn.summary())

/home/tasnim/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if sys.path[0] == '':
/home/tasnim/tensorflow/lib/python3.5/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [14]:
final_model.fit([X_train_1, X_train_0], y_train_1, validation_data=None, shuffle=False, epochs=1,
                                  verbose=0, batch_size=32, callbacks=[histories])

#print(history.history.keys())
#print(history.history['loss'])

In [16]:
for (i, loss) in enumerate(histories.losses):
    print("Iteration ",i, ":  ",loss)

Iteration  0 :   1.00089
Iteration  1 :   0.999261
Iteration  2 :   0.998147
Iteration  3 :   0.995858
Iteration  4 :   0.994788
Iteration  5 :   0.993319
Iteration  6 :   0.982336
Iteration  7 :   0.995565
Iteration  8 :   0.993602
Iteration  9 :   0.974734
Iteration  10 :   0.968557
Iteration  11 :   0.960383
Iteration  12 :   0.981107
Iteration  13 :   0.963892
Iteration  14 :   0.920721
Iteration  15 :   0.904191
Iteration  16 :   0.924583
Iteration  17 :   0.919438
Iteration  18 :   0.900229
Iteration  19 :   0.911221
Iteration  20 :   0.834518
Iteration  21 :   0.847019
Iteration  22 :   0.802103
Iteration  23 :   0.77268
Iteration  24 :   0.712854
Iteration  25 :   0.704299
Iteration  26 :   0.790785
Iteration  27 :   0.747592
Iteration  28 :   0.700453
Iteration  29 :   0.66265
Iteration  30 :   0.574695
Iteration  31 :   0.505085
Iteration  32 :   0.519072
Iteration  33 :   0.511181
Iteration  34 :   0.536189
Iteration  35 :   0.563653
Iteration  36 :   0.660626
Iteration  37 